## **Ring Signature Implementation (RSA Version)**

Based on the paper "How to leak a secret" by Ronald L. Rivest, Adi Shamir, and Yael Tauman



*   Install ***pycrypto*** module


In [ ]:
pip install pycrypto

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 446 kB 7.1 MB/s 
  Created wheel for pycrypto: filename=pycrypto-2.6.1-cp37-cp37m-linux_x86_64.whl size=499932 sha256=5a074abcbc42bd8fb56f29dba41939435d0735d52daa77aa21422e8c50228bae
  Stored in directory: /root/.cache/pip/wheels/cf/85/ba/bbd7c96add459de7598fb424e5ff2309baf2095c844ac0f191
Successfully built pycrypto




*   Import necessary modules



In [ ]:
import os
import hashlib              # Hash module
import random               # Module random dùng để lấy giá trị ngẫu nhiên
import Crypto.PublicKey.RSA # Implement RSA bằng module Crypto
import time                 # Module time dùng để đo runtime của hàm
import functools            # Functools để dùng các hàm hỗ trợ như map(), reduce(),...

- Each ring member holds a RSA public key: P = (n, e)
*   Define the Ring class and methods
* Symmetric Encryption Function reference to: https://gist.github.com/topcode4u/5988067


In [ ]:
class Ring:
    def __init__(self, k, L: int = 1024) -> None:
        self.k = k                                # Khởi tạo khóa k
        self.l = L
        self.n = len(k)                           # Số key (ring member)
        self.q = 1 << (L - 1)                     # Khi tính ra x của người ký sẽ có trường hợp > n của RSA, nếu lớn hơn thì khi chạy hàm verify sẽ không truy ngược lại x của người ký được

    def ring_sign(self, m: str, z: int):
        st = time.time()                          # start time

        self._hash(m)                             # k = H(m)
        x = [None] * self.n                       # x = [None, None,..., None], mảng rỗng dùng để chứa các giá trị x ngẫu nhiên thuộc {0,1}b
        u = random.randint(0, self.q)             # Chọn ngẫu nhiên giá trị khởi đầu (giá trị keo) u thuộc {0,1}b
        c = v = self._E(u)                        # Tạo biến chữ ký và biến tạm để dùng cho combining function

        first_range = list(range(z + 1, self.n))  # first range = [z + 1, ..., n -1]
        second_range = list(range(z))             # second range = [0, 1, 2, ..., z - 1]
        whole_range = first_range + second_range

        for i in whole_range:
            x[i] = random.randint(0, self.q)      # Chọn giá trị xi ngẫu nhiên cho các thành viên trong vòng
            y = self._g(x[i], self.k[i].e, self.k[i].n)
            v = self._E(v ^ y)                    
            if (i + 1) % self.n == 0:
                c = v

        # Tính x của người ký bằng reverse trapdoor function: xs = g^-1(ys)
        x[z] = self._g(v ^ u, self.k[z].d, self.k[z].n)

        et = time.time()                          # end time
        exec_time = et - st                       # execution time
        print('Executed in: ', exec_time*1000, 'ms')
        return [c] + x

    def ring_verify(self, m: str, X) -> bool:
        st = time.time()                          # start time
        self._hash(m)                             # k = H(m)

        # yi = gi(xi)
        def _f(i):
            return self._g(X[i + 1], self.k[i].e, self.k[i].n)

        y = map(_f, range(len(X) - 1))
        y = list(y)

        def _g(x, i):
            return self._E(x ^ y[i])
        # Verify nếu biểu thức vòng đúng bằng giá trị v
        r = functools.reduce(_g, range(self.n), X[0])
        et = time.time()                          # end time
        exec_time = et - st                       # execution time
        print('Verified in: ', exec_time*1000, 'ms')
        return r == X[0]

    def _hash(self, m):                         # Hàm băm H()
        msg = m.encode("utf-8")
        self.p = int(hashlib.sha256(msg).hexdigest(), 16)  # khóa k

    def _E(self, x):                            # Combining function
        msg = f"{x}{self.p}".encode("utf-8")
        return int(hashlib.sha256(msg).hexdigest(), 16)

    def _g(self, x, e ,n):                      # Trapdoor function
        q, r = divmod(x, n)
        if((q + 1) * n) <= ((1 << self.l) - 1): # Nếu (qi + 1)*ni <= 2^b
            result = q * n + pow(r, e, n)       # g(x) = q * n + f(r) với f(x) = x^e mod n
        else:
            result = x                          # Không thì trả về tham số đầu vào
        return result                           # Xem paper mục 3.1

- Main()

In [ ]:
size = 4                                        # Ring size
m1 = "This is a classified meessage"            # Message to be signed by signer
m2 = "This is not the message he signed"

# generate(key length, randfunc which returns random bytes)
def _rn(_):
    return Crypto.PublicKey.RSA.generate(1024, os.urandom)          

key = map(_rn, range(size))
key = list(key)                                  # Ring's public keys

r = Ring(key)

# Supposedly, we have a ring consists of 4 members named A, B, C, D followed by ID = [0, 1, 2, 3]. Since the anonymity of the signer is protected, only the actual signer themself knows that they signed the message
signer = random.randint(0, size)                

signature = r.ring_sign(m1, signer)               # Signing message m

if(r.ring_verify(m1, signature)):                 # Verify signature c on message m1
  print('Valid signature!')
else:
  print('Invalid signature!')       

#assert r.ring_verify(m2, signature), "Invalid signature!"               # Verify signature c on message m2


Executed in:  5.432605743408203 ms
Verified in:  0.3573894500732422 ms
Valid signature!


- Output signature

In [ ]:
print(signature)

[45602404720944561088275190248842823302607876551047547578053239410566361262685, 43104743072908202708647376912636233216510629903939388910719113530699639688984529079258386410011272391651233888514352047815826486416791635855360320534566728581148769218114077386439690622842440337988891984318865466070704692188172993416477712928133377630261960708874205479684355928219077745236337642258716164649, 5929325741118008273760646712026650201526708235590455893049375505157577866103090653230265601542831858145509412666591343643393645885597801372256908917782583461989286509779693771835575918652445557609982587336576157310301157518006745855469416076244430532392108339799495884301811598984254240041140656698738203579, 2096193279331186387139199815671049502213486078156218724387472429507219766282095564513319541863243545457375935728610052716564553914581035789628055428305771444318179923972621067903012993774862392193173842995098198473241549507175878493925716940053714844147225901577032660943977225358168227339235807324002